# 基于三轴加速度传感器Multilayer Perceptron（MLP）动作分类---改进

In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
df_data1 = pd.read_csv(r'UpStairsAccelerometer.csv')#数据使用“SensorRecord”采集android三轴加速度传感器信息
del df_data1['Milliseconds']
df_data1_5 = pd.read_csv(r'1.5LeftUpStairsAccelerometer.csv')
del df_data1_5['Milliseconds']
df_data11=pd.concat([df_data1,df_data1_5])
df_data11=df_data11.values#DataFrame2Numpy.Array               #1上楼梯
df_data2 = pd.read_csv(r'DownStairsAccelerometer.csv')
del df_data2['Milliseconds']
df_data2=df_data2.values#DataFrame2Numpy.Array                 #2下楼梯
df_data3 = pd.read_csv(r'FlatWalkAccelerometer.csv')
del df_data3['Milliseconds']
df_data3=df_data3.values#DataFrame2Numpy.Array                     #3平地走

df_data01 = pd.read_csv(r'StandAccelerometer.csv')
del df_data01['Milliseconds']
df_data02 = pd.read_csv(r'UpElevatorAccelerometer.csv')
del df_data02['Milliseconds']
df_data03 = pd.read_csv(r'DownElevatorAccelerometer.csv')
del df_data03['Milliseconds']
df_data00=pd.concat([df_data01,df_data02,df_data03])
df_data00=df_data00.values#DataFrame2Numpy.Array                         #0 其他，包括静止，上下电梯

In [4]:
print(df_data00.shape)
print(df_data11.shape)
print(df_data2.shape)

print(df_data3.shape)#each data gap is 10ms.
#there are 1000 ms in 1 sencond。
#10*10ms=100ms=0.1s，改进，让0.1s的数据来训练特征


(11814, 3)
(30308, 3)
(16933, 3)
(24142, 3)


In [6]:
import numpy as np
df_data00=np.delete(df_data00,list(range(4)),axis=0)#[0,1,2,3]
df_data11=np.delete(df_data11,list(range(8)),axis=0)
df_data2=np.delete(df_data2,list(range(3)),axis=0)
df_data3=np.delete(df_data3,list(range(2)),axis=0)
#11814=11810+4,
#30308=30300+8,
#16933=16930+3,
#24142=24140+2,
print(df_data00.shape)
print(df_data11.shape)
print(df_data2.shape)
print(df_data3.shape)

(11810, 3)
(30300, 3)
(16930, 3)
(24140, 3)


In [7]:
df_data00[[0,1,2],:]

array([[-1.7678986,  2.510666 ,  9.22197  ],
       [-1.8588257,  2.5872345,  9.006622 ],
       [-1.8995056,  2.682953 ,  9.150192 ]])

In [8]:
#去除几行数据 每10条数据合成为一条，让0.1s的数据来训练特征
df_data00.shape=(1181,30)
df_data11.shape=(3030,30)
df_data2.shape=(1693,30)
df_data3.shape=(2414,30)

In [9]:
df_data00[[0,1],:]

array([[-1.7678986 ,  2.510666  ,  9.22197   , -1.8588257 ,  2.5872345 ,
         9.006622  , -1.8995056 ,  2.682953  ,  9.150192  , -1.9138641 ,
         2.8600159 ,  9.659866  , -1.7487488 ,  2.7930145 , 10.351395  ,
        -1.3252258 ,  2.7068787 , 10.722275  , -0.9447632 ,  2.6661987 ,
        10.863449  , -0.4805603 ,  2.52742   , 10.954376  , -0.19102478,
         2.4077759 , 10.710312  ,  0.09851074,  2.6159515 , 10.224564  ],
       [-0.18145752,  2.5991974 ,  9.152588  , -0.86579895,  3.8434753 ,
         8.566345  , -0.7677002 ,  4.101898  ,  7.937027  ,  0.54356384,
         3.7214355 ,  7.5398254 , -0.3920288 ,  4.3914337 ,  8.355774  ,
        -0.14794922,  3.9966125 ,  8.93483   ,  0.1846466 ,  3.3792572 ,
         9.564148  ,  0.5076752 ,  3.1830597 ,  9.7795105 ,  0.34736633,
         3.221344  ,  9.683792  ,  0.07936096,  3.5611115 ,  9.296158  ]])

In [10]:
df_label00=np.zeros(1181)
df_label11=np.ones(3030)
df_label2=2*np.ones(1693)
df_label3=3*np.ones(2414)
print(df_label00.shape)
print(df_label11.shape)
print(df_label2.shape)
print(df_label3.shape)
df_label=np.append(np.append(np.append(df_label00,df_label11),df_label2),df_label3)
print(df_label.shape)

(1181,)
(3030,)
(1693,)
(2414,)
(8318,)


In [11]:
df_label

array([0., 0., 0., ..., 3., 3., 3.])

In [12]:
df_data=np.concatenate((np.concatenate((np.concatenate((df_data00,
                                                        df_data11),axis=0)
                                        ,df_data2),axis=0)
                        ,df_data3),axis=0)
print(df_data)

[[-1.7678986   2.510666    9.22197    ...  0.09851074  2.6159515
  10.224564  ]
 [-0.18145752  2.5991974   9.152588   ...  0.07936096  3.5611115
   9.296158  ]
 [-0.15513611  3.7740784   8.796051   ...  2.1922302   4.1114655
   9.1549835 ]
 ...
 [-1.1122589   4.582855    7.221573   ... -0.55952454  4.898712
   8.5448    ]
 [-0.88972473  4.9130554   7.965744   ... -0.42552185  6.3750763
   8.843903  ]
 [-0.4853363   6.140579    8.886978   ... -2.4091797   6.1740875
   8.209808  ]]


In [13]:
df_data.shape

(8318, 30)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.1)

In [15]:
x_train.shape[0]

7486

In [16]:
x_test.shape[0]

832

In [17]:
x_test.shape[0]/df_data.shape[0]

0.10002404424140418

In [18]:
outputpath="total30.csv"#其中：index是否要索引，header是否要列名，True就是需要
np.savetxt(outputpath, df_data, delimiter=",")
outputpath="totalLabel30.csv"
np.savetxt(outputpath, df_label, delimiter=",")

In [19]:
outputpath="x_train30.csv"#其中：index是否要索引，header是否要列名，True就是需要
np.savetxt(outputpath, x_train, delimiter=",")
outputpath="x_test30.csv"
np.savetxt(outputpath, x_test, delimiter=",")
outputpath="y_train30.csv"#其中：index是否要索引，header是否要列名，True就是需要
np.savetxt(outputpath, y_train, delimiter=",")
outputpath="y_test30.csv"
np.savetxt(outputpath,y_test, delimiter=",")

In [20]:
import time
time_start=time.time()
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_dim=30, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(4, activation='softmax')
])
#输入层3维样本X，Y，Z，神经元分别*个不同的层，然后随机失活dropout=0.2,最后计算结果
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#WARNING:tensorflow:Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>
#pd.DataFrame2Numpy
#优化器使用Adam优化算法，交叉熵损失函数，学习率0.001，精度评价模型
model.fit(x_train, y_train, batch_size=4, epochs=9)
#训练
time_end=time.time()
print('time cost',time_end-time_start,'s')

Train on 7486 samples
Epoch 1/9
7486/7486 [==============================] - 19s 3ms/sample - loss: 0.9510 - accuracy: 0.6030
Epoch 2/9
7486/7486 [==============================] - 13s 2ms/sample - loss: 0.8456 - accuracy: 0.6602
Epoch 3/9
7486/7486 [==============================] - 14s 2ms/sample - loss: 0.8096 - accuracy: 0.6667
Epoch 4/9
7486/7486 [==============================] - 13s 2ms/sample - loss: 0.7867 - accuracy: 0.6847
Epoch 5/9
7486/7486 [==============================] - 14s 2ms/sample - loss: 0.7787 - accuracy: 0.6885
Epoch 6/9
7486/7486 [==============================] - 14s 2ms/sample - loss: 0.7553 - accuracy: 0.6984
Epoch 7/9
7486/7486 [==============================] - 14s 2ms/sample - loss: 0.7406 - accuracy: 0.7008
Epoch 8/9
7486/7486 [==============================] - 13s 2ms/sample - loss: 0.7291 - accuracy: 0.7048
Epoch 9/9
7486/7486 [==============================] - 16s 2ms/sample - loss: 0.7167 - accuracy: 0.7119
time cost 132.77914547920227 s


In [21]:
 234.15752744674683/742.0994386672974#速度仅需要原方法的31.55%

0.3155338964644141

速度仅需要原方法的 234.15752744674683/742.0994386672974=31.55%

改进前的精确度accuracy: 0.6725，改进后的分类精确度accuracy: 0.7164。

可以说是又快又好，可以看出改进是有效的，可惜改进也是有限的。原因要么就是测量数据不够（也许可以无脑重复数据采集，大力出奇迹），要么是这个多层感知机模型解决这种三轴加速度传感器问题存在缺陷，如果是这样的话，也许该更换模型尝试得能否成功，比如支持向量机。

In [22]:
model.save('AcceleratometerSaveHDF5-30.h5')

In [23]:
x_test[0:3,:]

array([[ 9.397018 ,  2.11586  ,  1.4740143,  9.4664   ,  2.3288116,
         1.7467957,  9.5334015,  2.469986 ,  1.8975525,  9.562119 ,
         2.4771729,  1.9597626,  9.473587 ,  2.4915314,  1.9621582,
         9.2534485,  2.4939117,  1.9621582,  9.152954 ,  2.560913 ,
         2.019577 ,  9.040482 ,  2.6877441,  1.986084 ,  9.006989 ,
         2.7068787,  1.8664398,  8.918457 ,  2.7978058,  1.8090057],
       [ 7.96611  , -4.7635193,  3.0580597,  7.9828644, -4.775482 ,
         3.0867767,  7.9517517, -4.811371 ,  3.0987396,  7.937393 ,
        -4.799408 ,  3.0604553,  7.9230347, -4.844864 ,  3.0460968,
         7.9062805, -4.840088 ,  3.0389252,  7.9230347, -4.885544 ,
         3.0006409,  7.908676 , -4.9094696,  3.0317383,  7.8895416,
        -4.880768 ,  2.976715 ,  7.8823547, -4.880768 ,  2.9623566],
       [ 9.684158 ,  3.0346985,  1.9549713,  9.11467  ,  3.7549438,
         1.9310455,  8.698303 ,  3.8075714,  1.5458069,  8.258026 ,
         3.4199371,  1.2060242,  7.7938232,  2

In [24]:
y_test[0:3]#谁标签，理论上概率最大

array([3., 0., 3.])

In [25]:
model.predict(x_test[0:3,:])#结果是[0,1,2,3]分别对应的概率，四个数字中概率最大的分别对应是上面的y_test

array([[0.00308093, 0.61445475, 0.26944238, 0.11302198],
       [0.7898558 , 0.00239102, 0.06062512, 0.14712803],
       [0.00493979, 0.53617245, 0.34958768, 0.10930012]], dtype=float32)

In [26]:
model.evaluate(x_test[0:2333,:], y_test[0:2333])

832/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.7600615689387689, 0.69471157]

In [27]:
from tensorflow.keras.models import load_model
classifier = load_model('AcceleratometerSaveHDF5-30.h5') 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
classifier.evaluate(x_test[0:666,:], y_test[0:666])

666/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[0.7326904438637398, 0.7102102]

In [29]:
classifier.predict(x_test[0:3,:])#结果和前面一样

array([[0.00308093, 0.61445475, 0.26944238, 0.11302198],
       [0.7898558 , 0.00239102, 0.06062512, 0.14712803],
       [0.00493979, 0.53617245, 0.34958768, 0.10930012]], dtype=float32)

In [30]:
classifier.predict(df_data2)[0:100,:]#理论上应该全是2即第三个概率最大

array([[6.50782526e-01, 2.04452664e-01, 9.33990628e-02, 5.13657406e-02],
       [6.17119014e-01, 2.25960806e-01, 1.02866314e-01, 5.40538691e-02],
       [6.68547332e-01, 1.94302350e-01, 8.96567851e-02, 4.74935360e-02],
       [7.80683219e-01, 1.08029544e-01, 6.43827841e-02, 4.69045117e-02],
       [7.59772301e-01, 1.26807451e-01, 7.03544170e-02, 4.30658124e-02],
       [7.69044578e-01, 7.36018717e-02, 5.90567030e-02, 9.82968137e-02],
       [2.24641159e-01, 3.52847815e-01, 2.03751326e-01, 2.18759716e-01],
       [3.52175325e-01, 3.19468141e-01, 1.78887770e-01, 1.49468690e-01],
       [1.79497227e-01, 1.10278316e-01, 3.88194203e-01, 3.22030216e-01],
       [2.21511751e-01, 3.58097911e-01, 2.25424647e-01, 1.94965780e-01],
       [8.06502253e-02, 2.39236489e-01, 2.74497360e-01, 4.05615985e-01],
       [3.05328548e-01, 2.34986976e-01, 1.19281583e-01, 3.40402931e-01],
       [2.02842638e-01, 3.23119909e-01, 1.84279144e-01, 2.89758265e-01],
       [6.20167293e-02, 4.72891331e-01, 3.25381309e

还是有对有错，看着错的分类，有时候分外揪心。